# Simple `openrouteservice-py` examples

This notebook is just a small reference on how to use `openrouteservice` package to request:
- [directions](https://openrouteservice.org/dev)
- [isochrones](https://openrouteservice.org/dev/#/api-docs/v2/isochrones/{profile}/post)
- [matrix](https://openrouteservice.org/dev/#/api-docs/v2/matrix/{profile}/post)
- [POIs](https://openrouteservice.org/dev/#/api-docs/pois/post)
- [Elevation](https://openrouteservice.org/dev/#/api-docs/elevation/line/post)
- [Route Optimization](https://openrouteservice.org/dev/#/api-docs/optimization/post)

In [4]:
import openrouteservice as ors
import folium

client = ors.Client(key='5b3ce3597851110001cf62485a59fee9eaee45ca848c807ba6f97539')

## TOC

- **[Directions](#Directions)**
- **[Isochrones](#Isochrones)**
- **[Matrix](#Matrix)**
- **[POIs](#POIs)**
- **[Geocoding](#Geocoding)**
- **[Elevation](#Elevation)**
- **[Optimization](#Optimization)**

## Directions

In [5]:
m = folium.Map(location=[52.521861, 13.40744], tiles='cartodbpositron', zoom_start=13)

# Some coordinates in Berlin
coordinates = [[13.42731, 52.51088], [13.384116, 52.533558]]

route = client.directions(
    coordinates=coordinates,
    profile='foot-walking',
    format='geojson',
    options={"avoid_features": ["steps"]},
    validate=False,
)
folium.PolyLine(locations=[list(reversed(coord)) 
                           for coord in 
                           route['features'][0]['geometry']['coordinates']]).add_to(m)
    
m

## Isochrones

In [6]:
m = folium.Map(location=[52.521861, 13.40744], tiles='cartodbpositron', zoom_start=13)

# Some coordinate in Berlin
coordinate = [[13.384116, 52.533558]]

iso = client.isochrones(
    locations=coordinate,
    profile='foot-walking',
    range=[600, 1200],
    validate=False,
    attributes=['total_pop']
)

# Also create a Popup with Population count within isochrones
for isochrone in iso['features']:
    folium.Polygon(locations=[list(reversed(coord)) for coord in isochrone['geometry']['coordinates'][0]],
                  fill='00ff00',
                  popup=folium.Popup("Population: {} people".format(isochrone['properties']['total_pop'])),
                  opacity=0.5).add_to(m)
    
m

## Matrix

In [7]:
m = folium.Map(location=[52.521861, 13.40744], tiles='cartodbpositron', zoom_start=13)

# Some coordinates in Berlin
coordinates = [[13.384116, 52.533558], [13.428726, 52.519355], [13.41774, 52.498929], [13.374825, 52.496369]]

matrix = client.distance_matrix(
    locations=coordinates,
    profile='foot-walking',
    metrics=['distance', 'duration'],
    validate=False,
)

for marker in coordinates:
    folium.Marker(location=list(reversed(marker))).add_to(m)

print("Durations in secs: {}\n".format(matrix['durations']))
print("Distances in m: {}".format(matrix['distances']))

m

Durations in secs: [[0.0, 2754.05, 3795.19, 3660.45], [2754.05, 0.0, 1989.41, 4003.03], [3795.19, 1989.41, 0.0, 2467.21], [3660.45, 4003.03, 2467.21, 0.0]]

Distances in m: [[0.0, 3825.17, 5271.25, 5084.1], [3825.17, 0.0, 2763.16, 5559.91], [5271.25, 2763.16, 0.0, 3426.76], [5084.1, 5559.91, 3426.76, 0.0]]


## POIs

In [8]:
m = folium.Map(location=[52.521861, 13.40744], tiles='cartodbpositron', zoom_start=13)

# Some coordinates in Berlin
geojson = {"type": "point", "coordinates": [13.384116, 52.533558]}

pois = client.places(
    request='pois',
    geojson=geojson,
    buffer=500,  # searches within 500 meters of specified point
    filter_category_ids=[561, 568],  # https://github.com/GIScience/openpoiservice/blob/master/openpoiservice/server/categories/categories.yml
    validate=False,
#     dry_run=True
)

for poi in pois[0]['features']: 
    folium.Marker(
        location=list(reversed(poi['geometry']['coordinates'])),
        icon=folium.Icon(icon='info-sign'),
        popup=folium.Popup(poi['properties'].get('osm_tags', {}).get('name', "Bar ohne Namen")),
    ).add_to(m)
    
m

KeyError: 0

## Geocoding

### Forward Geocoding

In [9]:
m = folium.Map(location=[52.521861, 13.40744], tiles='cartodbpositron', zoom_start=13)

address = "Mollstraße 1, Berlin"

geocode = client.pelias_search(
    text=address,
    focus_point=list(reversed(m.location)),
    validate=False,
)

for result in geocode['features']:
    folium.Marker(
        location=list(reversed(result['geometry']['coordinates'])),
        icon=folium.Icon(icon='compass', color='green', prefix='fa'),
        popup=folium.Popup(result['properties']['name'])
    ).add_to(m)

m

### Reverse Geocoding

In [10]:
m = folium.Map(location=[52.521861, 13.40744], tiles='cartodbpositron', zoom_start=13)

coordinate = [13.413448, 52.490993]

reverse = client.pelias_reverse(
    point=coordinate,
    validate=False,
)

for result in reverse['features']:
    folium.Marker(
        location=list(reversed(result['geometry']['coordinates'])),
        icon=folium.Icon(icon='history', color='green', prefix='fa'),
        popup=folium.Popup(result['properties']['label'])
    ).add_to(m)

m

## Elevation

### Point

In [12]:
coordinate = [13.384116, 52.533558]

elevation = client.elevation_point(
    format_in='point',
    format_out='point',
    geometry=coordinate
)

print(elevation['geometry'])

[13.384115999999999, 52.533558, 34.0]


### Line

In [13]:
coordinates = [[13.384116, 52.533558], [13.428726, 52.519355], [13.41774, 52.498929], [13.374825, 52.496369]]

elevation = client.elevation_line(
    format_in='polyline',  # other options: geojson, encodedpolyline
    format_out='geojson',
    geometry=coordinates,
)

print(elevation['geometry']['coordinates'])

[[13.384116, 52.533558, 34], [13.428726, 52.519355, 40], [13.41774, 52.498929, 40], [13.374825, 52.496369, 39]]


## Optimization

### Simple waypoint optimization

With `openrouteservice-py` you can ask for waypoint optimization when requesting a normal `direction`. This assumes that your first coordinate is the start location and the last coordinate is the end location, i.e. only the `via` endpoints are optimized. To make it a round trip, make the first and last location the same.

In [14]:
m = folium.Map(location=[52.521861, 13.40744], tiles='cartodbpositron', zoom_start=13)

# Some coordinates in Berlin
coordinates = [[13.384116, 52.533558], [13.41774, 52.498929], [13.428726, 52.519355], [13.374825, 52.496369]]

# The popup will show the ID in the coordinate list. In a non-optimized waypoint order, the waypoints
# would have been visited from ID 0 to ID 3 sequentially.
for idx, coords in enumerate(coordinates):
    folium.Marker(location=list(reversed(coords)),
                 popup=folium.Popup("ID: {}".format(idx))).add_to(m)

route = client.directions(
    coordinates=coordinates,
    profile='foot-walking',
    format='geojson',
    validate=False,
    optimize_waypoints=True
)

folium.PolyLine(locations=[list(reversed(coord)) 
                           for coord in 
                           route['features'][0]['geometry']['coordinates']]).add_to(m)
    
m

### Optimize job scheduling for multiple vehicles

The next example makes more use of the power of the ORS optimization endpoint. Two vehicles are assigned to 6 jobs, where the capacity constraints are such that each vehicle can only carry out 3 jobs on its route.

In [15]:
m = folium.Map(location=[52.521861, 13.40744], tiles='cartodbpositron', zoom_start=13)

vehicle_locations = [[13.390446, 52.506087], [13.384116, 52.533558]]
job_locations = [[13.428726, 52.519355],
                 [13.41774, 52.498929],
                 [13.374825, 52.496369],
                 [13.378859, 52.509796],
                 [13.400488, 52.509691],
                 [13.358517, 52.524264]]

# Assign vehicles to do the jobs
vehicles = []
for idx, coords in enumerate(vehicle_locations):
    vehicles.append(ors.optimization.Vehicle(
        id=idx,
        profile='driving-car',
        start=coords,
        end=coords,
        capacity=[3]  # Limit capacity so only 3 jobs can be taken by each vehicle
    ))
    folium.Marker(location=list(reversed(coords)), icon=folium.Icon(icon='truck', prefix='fa')).add_to(m)

# Define jobs to be carried out
jobs=[]
for idx, coords in enumerate(job_locations):
    jobs.append(ors.optimization.Job(
        id=idx,
        location=coords,
        amount=[1]  # Occupies capacity in vehicle
    ))
    folium.Marker(location=list(reversed(coords)), icon=folium.Icon(icon='archive', prefix='fa', color='green')).add_to(m)

optimized = client.optimization(
    jobs=jobs,
    vehicles=vehicles,
    geometry=True,  ## will output the geometry,
)

folium.PolyLine(
    locations=[list(reversed(coords)) for coords in ors.convert.decode_polyline(optimized['routes'][0]['geometry'])['coordinates']],
    color='red'
).add_to(m)

folium.PolyLine(
    locations=[list(reversed(coords)) for coords in ors.convert.decode_polyline(optimized['routes'][1]['geometry'])['coordinates']],
    color='orange'
).add_to(m)

m